In [11]:
import os
import psycopg2
from dotenv import load_dotenv
import mysql.connector
import boto3

In [12]:

def import_from_S3(S3directory, filename, outputDirectory):

    # load environment variables from .env file
    load_dotenv()

    # access specific s3 bucket
    bucket_name = os.getenv('AWS_BUCKET')

    # create an S3 client
    s3 = boto3.client('s3')

    # list all of the buckets in your account
    response = s3.list_buckets()
    #print(response)


    print(S3directory + filename)

    s3.download_file(bucket_name, S3directory + filename, outputDirectory + filename)


In [13]:

load_dotenv()
host=os.getenv('DB_HOST')

conn = mysql.connector.connect(
    host=os.getenv('DB_HOST'),
    user=os.getenv('DB_USERNAME'),
    password= os.getenv('DB_PASSWORD'),
    database=os.getenv('DB_DATABASE'),
    port=os.getenv('DB_PORT')
)

cursor = conn.cursor()

query = "select url from logograb_videos where id = 1"

cursor.execute(query)

result = cursor.fetchall()
video_url = result[0][0]
filename = video_url.replace('http://spect8-static.s3.amazonaws.com/', '')
print(filename)

#import_from_S3('', filename, '.\\visua\\videos\\')



7801176_2021_05_09_VIF_RBK_1st_Half_3cdbc3.mp4


In [16]:
import_from_S3('video-analysis/', '1.json', '.\\visua\\analyses\\')


conn.close()

video-analysis/1.json


In [ ]:
# load environment variables from .env file
load_dotenv()

# access specific s3 bucket
bucket_name = os.getenv('AWS_BUCKET')

# create an S3 client
s3 = boto3.client('s3')

# list all of the buckets in your account
response = s3.list_buckets()
#print(response)

file_name = '/visua/20210927_TV2_Sportsnyhetene_2125.mp4'
local_file_path = '/home/ubuntu/.hkt/logodetect/data/videos/'

#response = s3.get_object(Bucket=bucket_name, Key='7801207_2021_05_27_VIK_MIF_2nd_Half_416444.mp4')

#os.makedirs(os.path.dirname(local_file_path))

s3.download_file(bucket_name, 'video-analysis/1.json', '.\\visua\\analyses\\1.json')

